# Update profiles for functional groups of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Read data from database
- Write static markdown pages for a Jekyll site

## Set-up
Load all the libraries we will need in this script:

In [1]:
import os
from pathlib import Path
import re
from datetime import datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs
import shutil
import yaml

Read configuration parameters for the connection to the current version of the database:

In [2]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

## Copy DAM between repos
Assumming  `repo one` has been updated recently, copy files to `repo two`:

In [3]:
repodir = Path(os.path.expanduser('~')) / 'proyectos' / 'typology-website'
repo_one = repodir  / 'typology-map-content' / 'assets' / 'uploads'
repo_two =  repodir / 'Ecosystem-profiles-comments' / 'assets' / 'diagrams'

DAMfiles=list()
for fn in os.listdir(repo_one):
    if fn.find('diagram')>0:
        DAMfiles.append(fn)

In [43]:
for infile in DAMfiles:
    nfn = infile.replace('-diagram.png','')
    outfile = re.sub(r'([mfts]+)_([0-9])_([0-9])',r'\1\2.\3',nfn).upper() +'.png'
    source =  repo_one / infile
    destination =  repo_two / outfile
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
    

## Connect to database

In [228]:
conn = psycopg2.connect(**params)
cur = conn.cursor(cursor_factory=DictCursor)


## Add map information to data folder

In [224]:
outfile =  repodir / 'Ecosystem-profiles-comments' / '_data' / 'references.yaml'
qry = """
SELECT ref_code,ref_cite
FROM ref_list 
WHERE ref_code IN (SELECT distinct ref_code from map_references) 
OR ref_code IN (SELECT distinct ref_code from efg_references);
"""
cur.execute(qry)
references = cur.fetchall()
records=dict()
for item in references:
    records[item['ref_code']]=item['ref_cite']

with open(outfile, 'w') as file:
    yaml.dump(records, file, allow_unicode=True, encoding = 'utf-8')


In [222]:
records['Abell et al. 2008']

'Abell R, Thieme ML, Revenga C, Bryer M, Kottelat M, Bogutskaya N, Coad B, Mandrak N, Contreras Balderas S, Bussing W, Stiassny MLJ, Skelton P, Allen GR, Unmack P, Naseka A, Ng R, Sindorf N, Robertson J, Armijo E, Higgins JV, Heibel TJ, Wikramanayake E, Olson D, López HL, Reis RE, Lundberg JG, Sabaj Pérez MH, Petry P (2008) *Freshwater ecoregions of the world: A new map of biogeographic units for freshwater biodiversity conservation*, **BioScience** 58: 403–414. DOI:[10.1641/B580507](https://doi.org/10.1641/B580507)'

In [211]:
outfile =  repodir / 'Ecosystem-profiles-comments' / '_data' / 'mapinfo.yaml'
qry="""
    SELECT map_code, map_version, code, map_source,contributors,status 
    FROM map_metadata WHERE status IN ('valid','replaced','superceded','superceeded') 
    ORDER BY map_version DESC;
    """
cur.execute(qry)
mapinfo = cur.fetchall()
records=dict()
record=dict()
record['code']='MISSING'
record['version']='MISSING'
record['description']='Map information is missing'
record['contributors']='MISSING'
records['MISSING']=record
for item in mapinfo:
    mapcode="%s_%s" % (item['map_code'],item['map_version'])
    record=dict()
    record['code']=item['map_code']
    record['efg']=item['code']
    record['version']=item['map_version']
    record['description']=item['map_source']
    record['contributors']=", ".join(item['contributors'])
    if item['status']=='valid':
        record['status']='valid'
    else:
        record['status']='replaced'
    records[mapcode]=record

with open(outfile, 'w') as file:
    yaml.dump(records, file)


In [229]:
outfile =  repodir / 'Ecosystem-profiles-comments' / '_data' / 'maprefs.yaml'
qry="""
    SELECT map_code,map_version,ARRAY_AGG(ref_code) as refs, ARRAY_AGG(dataset) as datasets
    FROM map_references 
    GROUP BY map_code, map_version;
    """
cur.execute(qry)
refs = cur.fetchall()

records=dict()
records['MISSING']={'references':'Missing references'}
for item in refs:
    mapcode="%s_%s" % (item['map_code'],item['map_version'])
    records[mapcode]={'references':item['refs'],
                      'datasets':item['datasets']}

with open(outfile, 'w') as file:
    yaml.dump(records, file)

In [230]:
outfile =  repodir / 'Ecosystem-profiles-comments' / '_data' / 'efgrefs.yaml'
qry="""
    SELECT code,ARRAY_AGG(ref_code) as refs 
    FROM efg_references 
    GROUP BY CODE
    """
cur.execute(qry)
refs = cur.fetchall()

records=dict()
records['MISSING']=('Missing references',)
for item in refs:

    records[item['code']]=item['refs']

with open(outfile, 'w') as file:
    yaml.dump(records, file)

## Update text content to version 2.1

In [195]:
qry="SELECT distinct code FROM functional_groups;"
cur.execute(qry)
efgs = cur.fetchall()


In [196]:
len(efgs)

110

In [168]:
sections = ("Ecological Traits","Key Ecological Drivers","Distribution")
version = 'v2.1'
records=list()
for efg in efgs:
    qry="""
    SELECT code,f.biome_code,f.name,b.name as biome_name,realms,f.update as original_date 
    FROM functional_groups f
    LEFT JOIN biomes b
        ON b.biome_code=f.biome_code
    WHERE code = %s
    ;"""
    cur.execute(qry,(efg[0],))

    efginfo = cur.fetchone()
    record=dict()
    
    record['name']=efginfo['name']
    record['biome']=efginfo['biome_name']
    record['realm']=efginfo['realms']
    record['code']=efginfo['code']
    record['biomecode']=efginfo['biome_code']
    record['contributors']=list()
    record['mapcontributors']=list()
    
    for section in sections:
        qry ="""
        SELECT description,contributors,version,update 
        FROM efg_%s 
        WHERE code = %s AND language = 'en' AND version=%s
        ORDER BY update DESC;
        """
        cur.execute(qry,(AsIs(section.lower().replace(' ','_')),efginfo['code'],version))
        sectinfo = cur.fetchone()
        record[section]=sectinfo['description']
        for author in sectinfo['contributors']:
            if author not in record['contributors']:
                record['contributors'].append(author)
        record['version']="%s (%s)" % (sectinfo['version'],sectinfo['update'].date())

    reflist1=list()
    reflist2=list()

    record['mapcontributorstr']=''
    record['MAPtext1']=''
    record['MAPtext2']=''
    record['maprefstr']='MAP INFO IS MISSING'
    record['Map description']='MAP INFO IS MISSING'
    
    qry="""
    SELECT map_code, map_source,contributors,map_version 
    FROM map_metadata WHERE code = %s AND status='valid' AND map_type='Indicative Map' 
    ORDER BY map_version DESC;
    """
    cur.execute(qry,(efg[0],))
    mapinfo = cur.fetchone()
    if mapinfo is not None:
        for author in mapinfo['contributors']:
            if author not in record['mapcontributors']:
                record['mapcontributors'].append(author)
        record['Map description']=mapinfo['map_source']
        record['mapversion']="%s_%s" % (mapinfo['map_code'],mapinfo['map_version'])
        if record['mapcontributors'] is not None:
            record['mapcontributorstr']=", ".join(record['mapcontributors'])
        else:
            record['mapcontributorstr']=''
        
        qry="""
    SELECT ref_cite  
    FROM map_references as e 
    LEFT JOIN ref_list as l ON e.ref_code=l.ref_code 
    WHERE map_code = %s AND map_version = %s
    ORDER BY ref_cite;
    """
        cur.execute(qry,(mapinfo['map_code'],mapinfo['map_version']))
        refs = cur.fetchall()
        for ref in refs:
            reflist2.append("* "+ref[0])

        record['MAPtext1']='{% capture map_det %}'
        record['MAPtext2']='{% endcapture %}\n{% include MAP.html %}'
        record['maprefstr']="\n".join(reflist2)

    record['realmstr']=", ".join(record['realm'])
    record['contributorstr']=", ".join(record['contributors'])
    record['DAMtext']='{% include DAM.html %}'
    
    qry="""
    SELECT ref_cite 
    FROM efg_references as e 
    LEFT JOIN ref_list as l ON e.ref_code=l.ref_code 
    WHERE code = %s
    ORDER BY ref_cite;
    """
    cur.execute(qry,(efg[0],))
    refs = cur.fetchall()
    
    for ref in refs:
        if ref[0] is not None:
            reflist1.append("* "+ref[0])
    
    
    
    record['mainrefstr']="\n".join(reflist1)
    
    records.append(record)

In [169]:
template = """---
name: {name}
biome: {biome}
realm: {realmstr}
code: {code}
biomecode: {biomecode}
contributors: {contributorstr}
mapcontributors: {mapcontributorstr}
version: {version}
---
# Ecosystem properties

{Ecological Traits}

# Ecological Drivers

{Key Ecological Drivers}

# Diagramatic assembly model

{DAMtext}

# Distribution

{Distribution}

{MAPtext1}
{Map description}
{MAPtext2}

## References
### Main References
{mainrefstr}
### Map References
{maprefstr}
"""

In [170]:
for record in records:
    outfile =  repodir / 'Ecosystem-profiles-comments' / '_EFGs' / (record['code'] + '.md')
    outtext=template.format(**record)
    with open(outfile,'w') as f:
        f.write(outtext)

In [197]:
sections = ("Ecological Traits","Key Ecological Drivers","Distribution")
version = 'v2.1'
records=list()
for efg in efgs:
    qry="""
    SELECT code,f.biome_code,f.name,b.name as biome_name,realms,f.update as original_date 
    FROM functional_groups f
    LEFT JOIN biomes b
        ON b.biome_code=f.biome_code
    WHERE code = %s
    ;"""
    cur.execute(qry,(efg[0],))

    efginfo = cur.fetchone()
    record=dict()
    
    record['name']=efginfo['name']
    record['biome']=efginfo['biome_name']
    record['realm']=efginfo['realms']
    record['code']=efginfo['code']
    record['biomecode']=efginfo['biome_code']
    record['contributors']=list()
    
    for section in sections:
        qry ="""
        SELECT description,contributors,version,update 
        FROM efg_%s 
        WHERE code = %s AND language = 'en' AND version=%s
        ORDER BY update DESC;
        """
        cur.execute(qry,(AsIs(section.lower().replace(' ','_')),efginfo['code'],version))
        sectinfo = cur.fetchone()
        record[section]=sectinfo['description']
        for author in sectinfo['contributors']:
            if author not in record['contributors']:
                record['contributors'].append(author)
        record['version']="%s (%s)" % (sectinfo['version'],sectinfo['update'].date())

    reflist1=list()
    reflist2=list()

    
    qry="""
    SELECT map_code, map_version 
    FROM map_metadata WHERE code = %s AND status='valid' AND map_type='Indicative Map' 
    ORDER BY map_version DESC;
    """
    cur.execute(qry,(efg[0],))
    mapinfo = cur.fetchone()
    if mapinfo is not None:
        record['mapcode']="%s_%s" % (mapinfo['map_code'],mapinfo['map_version'])
    else:
        record['mapcode']="MISSING" 


    record['realmstr']=", ".join(record['realm'])
    record['contributorstr']=", ".join(record['contributors'])
    record['DAMtext']='{% include DAM.html %}'
    
    qry="""
    SELECT ref_cite 
    FROM efg_references as e 
    LEFT JOIN ref_list as l ON e.ref_code=l.ref_code 
    WHERE code = %s
    ORDER BY ref_cite;
    """
    cur.execute(qry,(efg[0],))
    refs = cur.fetchall()
    
    for ref in refs:
        if ref[0] is not None:
            reflist1.append("* "+ref[0])
    
    
    #record['maprefstr']='{% for ref in map.contributors %}\n* {{ref}}\n{% endfor %}'
    
    #record['mainrefstr']="\n".join(reflist1)
    
    records.append(record)

In [198]:
template = """---
name: {name}
biome: {biome}
realm: {realmstr}
code: {code}
biomecode: {biomecode}
contributors: {contributorstr}
version: {version}
mapcode: {mapcode}
---
# Ecosystem properties

{Ecological Traits}

# Ecological Drivers

{Key Ecological Drivers}

# Diagramatic assembly model

{DAMtext}

# Distribution

{Distribution}

"""

In [199]:
len(records)

110

In [200]:
for record in records:
    outfile =  repodir / 'Ecosystem-profiles-comments' / '_EFGs' / (record['code'] + '.md')
    outtext=template.format(**record)
    with open(outfile,'w') as f:
        f.write(outtext)

In [188]:
record
#print(outtext)

{'name': 'TF1.1 Tropical flooded forests and peat forests',
 'biome': 'TF1. Palustrine wetlands biome',
 'realm': ['Terrestrial', 'Freshwater'],
 'code': 'TF1.1',
 'biomecode': 'TF1',
 'contributors': ['DA Keith',
  'RT Kingsford',
  'R Mac Nally',
  'KM Rodriguez-Clark',
  'A Etter'],
 'mapcontributors': [],
 'Ecological Traits': 'Closed-canopy forests in tropical swamps and riparian zones have high biomass and LAI, with unseasonal growth and reproductive phenology. The canopy foliage is evergreen, varying in size from mesophyll to notophyll with moderate SLA. Productivity differs markedly between high-nutrient ‘white water’ riparian systems and low-nutrient ‘black water’ systems. In the latter, most of the nutrient capital is sequestered in plant biomass, litter, or peat, whereas in white water systems, soil nutrients are replenished continually by fluvial subsidies. Some trees have specialised traits conferring tolerance to low-oxygen substrates, such as surface root mats, pneumatop

## Add maps with version codes

## Add map descriptions

## Close database connection

In [227]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
